In [1]:
import json
import pandas as pd
import numpy as np
import re
import os
import requests
import dotenv

# Load dotenv
dotenv.load_dotenv()

# Get enviroment variables
# ========================
MAPBOX_KEY = os.getenv("MAPBOX_KEY")

# Create API key for GeoJSON in https://rapidapi.com/VanitySoft/api/boundaries-io-1/
# Then copy and paste the key in .env file
GEOJSON_KEY = os.getenv("GEOJSON_KEY")

# Load Crime Data

In [2]:
# Read CSV file to pandas dataframe
df = pd.read_csv('../crime/KCPD_Crime_Data_Complete.csv')

# Datatypes of columns
"{:,} rows in crime dataset..".format(df.shape[0])

/Users/mrnino/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,9,10,11,12,15,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'1,550,285 rows in crime dataset..'

In [3]:
# Remove rows with non-numeric zip codes
df = df[df['Zip Code'].apply(
  lambda x: True if type(x) in [float, int] else re.match(r'^-?\d+(?:\.\d+)?$', x) is not None)
]

# Ensure 5 digit Zip Codes. Convert zip code to string and fill with zeros
df['Zip Code'] = df['Zip Code'].fillna(0).astype(float).astype(int).astype(str).str.zfill(5)

crime_zipcodes = df['Zip Code'].unique()

"{} unique Zip Codes and {:,} rows in Crime Data".format(len(crime_zipcodes), df.shape[0])

'649 unique Zip Codes and 1,550,284 rows in Crime Data'

# Obtain GeoJSON for Kansas City from RapidAPI
For testing ZIP codes, open [https://kepler.gl/demo](https://kepler.gl/demo) and open the geojson file. 

In [4]:
url = "https://vanitysoft-boundaries-io-v1.p.rapidapi.com/reaperfire/rest/v1/public/boundary/zipcode/location"

querystring = {
    # Center of Kansas City
    "latitude": "39.099225",
    "longitude": "-94.5839147",
    # Radius of search (45 miles)
    "radius":"45",
    "showDetails":"true"
}

headers = {
    'x-rapidapi-host': "vanitysoft-boundaries-io-v1.p.rapidapi.com",
    'x-rapidapi-key': GEOJSON_KEY
    }

zipcodes_45mi_from_KC = requests.request("GET", url, headers=headers, params=querystring)

# Convert to JSON
with open('../map/zipcodes_45mi_clean.geojson', 'w') as f:
    f.write(zipcodes_45mi_from_KC.text)

### Alternative plan. Obtain data for all ZIP codes in crime data. 

In [5]:
import requests

url = "https://vanitysoft-boundaries-io-v1.p.rapidapi.com/rest/v1/public/boundary/zipcode"

querystring = {"combine":"false"}

payload = list(crime_zipcodes)
headers = {
    'content-type': "application/json",
    'x-rapidapi-host': "vanitysoft-boundaries-io-v1.p.rapidapi.com",
    'x-rapidapi-key': GEOJSON_KEY
    }

crime_geojson_api = requests.request("POST", url, data=json.dumps(payload), headers=headers, params=querystring)

with open('../map/zipcodes_in_crimeKC.geojson', 'w') as f:
    f.write(crime_geojson_api.text)



In [6]:
# Parse JSON response
data = json.loads(zipcodes_45mi_from_KC.text) 

# Write all zip codes to file
with open('../map/list_zipcodes_KansasCity.txt', 'w') as f:
    f.write('\n'.join([
    feature['properties']['zipCode']
    for feature in data['features']
]))


### Proportion of crimes within 45 miles range

In [7]:

# Filter geojson to only include zip codes with crime data
zip_codes_with_crime = [
    feature['properties']['zipCode']
    for feature in data['features']
    if feature['properties']['zipCode'] in crime_zipcodes
]

print("{}/{} zip codes with crime data within Kansas City".format(len(zip_codes_with_crime), len(crime_zipcodes)))

171/649 zip codes with crime data within Kansas City


In [8]:
# Count number of crimes per zip code
crime_by_zipcode = df.groupby('Zip Code').size().reset_index(name='count').sort_values('count', ascending=False)
crime_by_zipcode['with_zip_code'] = crime_by_zipcode['Zip Code'].isin(zip_codes_with_crime) 

print("{0:10.1f}% of crimes with ZIP code within Kansas City".format( 
  100 * sum(crime_by_zipcode[crime_by_zipcode['with_zip_code'] == True]['count']) / sum(crime_by_zipcode['count']))
)


# If you want to see that in Excel, you can use this code
# crime_by_zipcode.to_csv('../crime/crimes_count_by_zipcode.csv')

      95.2% of crimes with ZIP code within Kansas City


# Prepare crime data for map

In [9]:
# Dictionary of zip codes and count of crimes
zip_code_counts = {
    zipcode: count
    for zipcode, count in zip(crime_by_zipcode['Zip Code'], crime_by_zipcode['count'])
}

# Add count of crimes to geojson
for feature in data['features']:
    zipcode = feature['properties']['zipCode']
    feature['properties']['crime_count'] = zip_code_counts.get(zipcode, 0)

with open('../map/45mi_with_crimes.geojson', 'w') as f:
    json.dump(data, f)

# Work in Progress. Extract lat/lon from LOCATION

In [10]:
import datetime

In [11]:
# Extract latitude and longitude from address text usign regex
def extract_lat_long(address):
    # Match lat and long
    match = re.search(r'\((\d+\.\d+),*\s*(-?\d+\.\d+)\)', str(address))
    if match:
        return float(match.group(1)), float(match.group(2))
    
    # Match long and lat
    match_inv = re.search(r'\((-?\d+\.\d+),*\s*(\d+\.\d+)\)', str(address))
    if match_inv:
        return float(match_inv.group(2)), float(match_inv.group(1))
    
    # No match
    return None, None
        

# Add latitude and longitude to crime data
df['latitude'] = df['Location'].apply(extract_lat_long).apply(lambda x: x[0])
df['longitude'] = df['Location'].apply(extract_lat_long).apply(lambda x: x[1])


In [12]:

zipcodes_in_map = [feature['properties']['zipCode'] for feature in data['features']]
with_location = df[df['Zip Code'].isin(zipcodes_in_map) & (df['latitude'].notnull() & df['longitude'].notnull())]

# Calculate average latitude and longitude by zip code
zip_code_center = with_location.groupby('Zip Code').agg({
    'latitude': 'median',
    'longitude': 'median'
}).reset_index().rename(columns={
    'latitude': 'zc_latitude',
    'longitude': 'zc_longitude'
})

zip_code_center

,Zip Code,zc_latitude,zc_longitude
0,64011,38.968902,-94.066107
1,64012,38.828092,-94.513432
2,64013,39.072736,-94.552790
3,64014,39.017471,-94.415875
4,64015,39.017540,-94.278074
...,...,...,...
159,66213,38.898831,-94.702377
160,66214,38.943920,-94.584399
161,66215,38.938260,-94.752552
162,66223,39.009104,-94.509520


In [13]:
# Convert reported date and time to datetime
df['datetime'] = pd.to_datetime(df[['Reported_Date', 'Reported Time']]
                   .astype(str).apply(' '.join, 1), errors='coerce' )

df[['Reported_Date', 'Reported Time', 'datetime']]

,Reported_Date,Reported Time,datetime
0,10/6/2009 0:00,3:24,2009-10-06 03:24:00
1,2/5/2009 0:00,11:45,2009-02-05 11:45:00
2,6/18/2009 0:00,22:50,2009-06-18 22:50:00
3,1/28/2009 0:00,18:44,2009-01-28 18:44:00
4,2/5/2009 0:00,11:45,2009-02-05 11:45:00
...,...,...,...
1550280,10/23/2021,8:34,2021-10-23 08:34:00
1550281,10/26/2021,18:28,2021-10-26 18:28:00
1550282,10/28/2021,14:43,2021-10-28 14:43:00
1550283,10/30/2021,11:03,2021-10-30 11:03:00


In [14]:
# Filter crimes by report date in last year
last_year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
# last_year = df[df['datetime'] >= last_year_ago]

# Print percentage with location
n_rows_ori = len(last_year)
n_rows_with_location = len(last_year[(last_year['latitude'].notnull() & last_year['longitude'].notnull())])
percent_with_location = round(n_rows_with_location / n_rows_ori * 100, 1)
print("{}% with location data in last year".format(percent_with_location))

# Fill missing latitude and longitude with zip code center
last_year = last_year.merge(zip_code_center, on='Zip Code', how='left')
last_year.loc[last_year['latitude'].isnull(), 'latitude'] = last_year['zc_latitude']
last_year.loc[last_year['longitude'].isnull(), 'longitude'] = last_year['zc_longitude']

# Filter rows with missing latitude or longitude
last_year = last_year[(last_year['latitude'].notnull() & last_year['longitude'].notnull())]
n_rows_with_location = len(last_year)
percent_with_location = round(n_rows_with_location / n_rows_ori * 100, 1)
print("{}% with location data after filling with zip code center".format(percent_with_location))

76.8% with location data in last year
98.8% with location data after filling with zip code center


In [15]:
# Select latitude and longitude columns
last_year = last_year[['datetime', 'Zip Code', 'Firearm Used Flag', 'latitude', 'longitude']]
last_year

,datetime,Firearm Used Flag,latitude,longitude
0,2020-12-19 09:42:00,False,39.106855,-94.540444
2,2020-12-22 04:49:00,False,39.085071,-94.581577
3,2020-12-23 12:22:00,False,39.269786,-94.471386
4,2020-12-04 15:54:00,False,39.033322,-94.545707
6,2020-12-09 20:30:00,False,39.067404,-94.538575
...,...,...,...,...
84037,2021-10-31 17:18:00,False,39.234274,-94.665822
84038,2021-10-23 08:34:00,False,39.098581,-94.582150
84039,2021-10-26 18:28:00,False,39.137175,-94.512021
84041,2021-10-30 11:03:00,False,39.112165,-94.522384


# En lugar de escribir a un .CSV .. escribir en la base de datos

In [16]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
last_year.to_csv('../crime/test_latlon.csv', index=False)

In [17]:


df.loc[df['Firearm Used Flag'].isin(['Y', 'True']), 'Firearm Used Flag'] = True
df.loc[df['Firearm Used Flag'].isin(['N', 'False']), 'Firearm Used Flag'] = False
df['Firearm Used Flag'].unique()

array([False, True], dtype=object)

In [18]:
# Number of crimes per zip code
df[df['Zip Code'].isin(zipcodes_in_map)].pivot_table(index='Zip Code', columns='Firearm Used Flag', values='Reported_Date', aggfunc='count')

Firearm Used Flag,False,True
Zip Code,,
64011,10.0,NaN
64012,132.0,NaN
64013,7.0,2.0
64014,20.0,NaN
64015,110.0,NaN
...,...,...
66213,9.0,NaN
66214,2.0,NaN
66215,19.0,NaN
